In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import hvplot.pandas
from scipy.stats import linregress
import json

In [9]:
file_path = "../Resources/VAERSDATA1 (3).csv"
covid_df = pd.read_csv(file_path)
covid_df.sample(20)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,SEX,DIED,DATEDIED,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,V_ADMINBY,VAX_MANU,HISTORY,ALLERGIES
22033,2085336,2/3/2022,TX,11,F,NaN,NaN,U,6/2/2021,2/1/2022,244.0,PVT,PFIZER\BIONTECH,NaN,NaN
160273,1444376,7/2/2021,MA,30,M,NaN,NaN,Y,4/28/2021,4/29/2021,1.0,PVT,MODERNA,Hashimoto's Thyroiditis and associated hypothy...,Amoxicillin and Hazelnuts
778700,1081314,3/8/2021,TN,74,F,NaN,NaN,Y,2/20/2021,2/20/2021,0.0,PVT,PFIZER\BIONTECH,Stage 3 kidney disease.,"Tree nuts, latex, metals"
875153,1876085,11/17/2021,MO,88,F,NaN,NaN,N,3/2/2021,11/5/2021,248.0,PVT,PFIZER\BIONTECH,diabetes,unknown
485769,1818734,10/26/2021,AL,54,M,NaN,NaN,N,10/23/2021,10/23/2021,0.0,PHM,MODERNA,NaN,"Flu vaccine, Pneumonia vaccine and food allerg..."
443224,1791273,10/16/2021,IL,51,F,NaN,NaN,N,10/7/2021,10/7/2021,0.0,PHM,PFIZER\BIONTECH,Medical History/Concurrent Conditions: Blood p...,NaN
217676,977102,1/27/2021,TX,35,F,NaN,NaN,Y,1/19/2021,1/20/2021,1.0,PVT,MODERNA,NaN,Penicillin and acetaminophen
366011,958754,1/20/2021,NJ,46,F,NaN,NaN,N,1/13/2021,1/20/2021,7.0,SCH,MODERNA,Asthmatic since age 2,Sulfa drugs
764678,1023447,2/11/2021,FL,73,F,NaN,NaN,N,1/25/2021,1/26/2021,1.0,PUB,PFIZER\BIONTECH,Medical History/Concurrent Conditions: Diabete...,NaN
309514,2104427,2/11/2022,TX,41,F,NaN,NaN,Y,4/1/2021,4/1/2021,0.0,UNK,MODERNA,NaN,NaN


In [10]:
covid_df.drop(['VAERS_ID', 'NUMDAYS', 'V_ADMINBY', 'HISTORY', 'ALLERGIES'], axis=1, inplace=True)

covid_df.head()


,RECVDATE,STATE,AGE_YRS,SEX,DIED,DATEDIED,RECOVD,VAX_DATE,ONSET_DATE,VAX_MANU
0,2/10/2021,IL,32,F,NaN,NaN,N,1/27/2021,2/4/2021,MODERNA
1,2/14/2021,NH,61,M,NaN,NaN,U,2/14/2021,2/14/2021,MODERNA
2,2/28/2021,WA,74,M,NaN,NaN,N,2/24/2021,2/24/2021,PFIZER\BIONTECH
3,3/2/2021,AZ,81,M,NaN,NaN,N,2/12/2021,2/26/2021,PFIZER\BIONTECH
4,4/16/2021,TX,15,M,NaN,NaN,Y,4/15/2021,4/15/2021,PFIZER\BIONTECH


In [13]:
# Adverse Reactions
adverse_reactions_df = covid_df[(covid_df['RECOVD'] != 'Y')]

# Seperate deaths
deaths_df = covid_df[(covid_df['DIED'] == 'Y')]

# Group by manufacturer for adverse reactions
adverse_reactions_by_manufacturer = adverse_reactions_df.groupby('VAX_MANU').size().reset_index(name='Adverse_Reactions')

# Group by manufacturer for deaths
deaths_by_manufacturer = deaths_df.groupby('VAX_MANU').size().reset_index(name='Deaths')

# Merge both summaries
summary_df = adverse_reactions_by_manufacturer.merge(deaths_by_manufacturer, on='VAX_MANU', how='outer').fillna(0)

# Print
summary_df



,VAX_MANU,Adverse_Reactions,Deaths
0,JANSSEN,35521,1317.0
1,MODERNA,268139,7563.0
2,NOVAVAX,218,0.0
3,PFIZER\BIONTECH,287050,7403.0
4,UNKNOWN MANUFACTURER,13543,623.0
